In [89]:
pip install pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [90]:
  !pip install -qU \
  transformers==4.31.0 \
  sentence-transformers==2.2.2 \
  datasets==2.14.0 \
  accelerate==0.21.0 \
  einops==0.6.1 \
  langchain==0.0.240 \
  xformers==0.0.20 \
  bitsandbytes==0.41.0\
  InstructorEmbedding

In [91]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [92]:
!pip install --user mysql-connector-python

In [93]:
pip install ctransformers

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chat_models import ChatOpenAI
from transformers import AutoTokenizer
from dotenv import load_dotenv
import os
import timeit
import sys
#import mysql.connector

C:\Users\navne\AppData\Roaming\Python\Python310\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY','1acea4e0-c639-4577-b431-6d607796e5d5')
PINECONE_API_ENV=os.environ.get('PINECONE_API_ENV', 'gcp-starter')

In [82]:
mysql_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    password="N090603",
    database="railway"  # Change this to your database name
)

NameError: name 'mysql' is not defined

In [97]:
mysql_cursor = mysql_connection.cursor()

In [98]:
mysql_query = "SELECT * FROM rschedule" 

In [99]:
mysql_cursor.execute(mysql_query)

In [100]:
rows = mysql_cursor.fetchall()

In [101]:
mysql_cursor.close()
mysql_connection.close()

In [102]:
file_path = 'railway_schedule.txt'

# Open the file for writing (create it if it doesn't exist)
with open(file_path, 'w') as file:
    # Iterate through the rows of data and write each row as text
    for row in rows:
        # Convert the row to a string (adjust the formatting as needed)
        row_text = f"Train No: {row[0]}, Train Name: {row[1]}, SEQ: {row[2]}, Station Code: {row[3]}, Station Name: {row[4]}, Arrival Time: {row[5]},Departure Time: {row[6]},Distance: {row[7]},Source Station: {row[8]},Source Station Name: {row[9]},Destination Station: {row[10]},Destination Station Name: {row[11]}"

        # Write the row to the file
        file.write(row_text)

# Close the file
file.close()

In [103]:
documents = []

# Open the file for reading
with open(file_path, 'r') as file:
    # Read each line (document) from the file
    for line in file:
        # Append the line (document) to the list of documents, removing leading/trailing whitespace
        documents.append(line.strip())

In [104]:
input_text = "\n".join(documents)
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
tokenized_text = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512, padding="max_length")
input_ids = tokenized_text["input_ids"]
input_text = tokenizer.decode(input_ids[0])

In [105]:
# Assuming 'documents' is a list of strings from the text file

# Initialize a list to store the split documents
split_docs = []

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

# Split each line/document into smaller chunks
for document in documents:
    split_chunks = text_splitter.split_text(input_text)
    split_docs.extend(split_chunks)

# Now, 'split_docs' contains the smaller text chunks


In [3]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

C:\Users\navne\AppData\Roaming\Python\Python310\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [4]:
pinecone.init(api_key=PINECONE_API_KEY,
              environment=PINECONE_API_ENV)

index_name="railway-chatbot"

In [5]:
#docsearch = Pinecone.from_texts(split_docs, embeddings, index_name=index_name)
docsearch=Pinecone.from_existing_index(index_name, embeddings)

In [6]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that I don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])

In [7]:
chain_type_kwargs={"prompt": PROMPT}

llm=CTransformers(model="llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [8]:
qa=RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever(search_kwargs={'k': 1}),return_source_documents=True, chain_type_kwargs=chain_type_kwargs)

In [10]:
#ASR
import speech_recognition as sr

import sys

#sys.stdin.reconfigure(encoding='utf-8')
#sys.stdout.reconfigure(encoding='utf-8')

# Initialize the recognizer
recognizer = sr.Recognizer()

# Use a microphone as the audio source
with sr.Microphone() as source:
    print("Listening...")
    
    # Adjust for ambient noise
    recognizer.adjust_for_ambient_noise(source)
    
    # Listen to the user's speech
    audio = recognizer.listen(source,timeout=10)

    print("Processing...")

try:
    # Recognize speech in Kannada
    language = "kannada"
    if language=="kannada":
        lang="kn-IN"
    elif language=="hindi":
        lang="hi-IN"
    else:
        lang="en"
    text = recognizer.recognize_google(audio, language=lang)

    # Print the recognized text
    print("you said:", text)

except sr.UnknownValueError:
    print("Error occured")

except sr.RequestError as e:
    print(f"Error occured; {e}")

Listening...
Processing...
you said: ಟ್ರೈನ್ ನಂಬರ್ 128 ಯಾವಾಗ ಬರುತ್ತದೆ


In [11]:
import googletrans
from googletrans import Translator

trans = Translator()

if lang=="kn-IN":
    lang="kn"

elif lang=="hi-IN":
    lang="hi"

if lang=="kn":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

elif lang=="hi":
    target_lang = "en"
    text = trans.translate(text, src=lang, dest=target_lang).text

print(text)

When does the train number 128 come


In [12]:
user_input=text
if user_input=='exit':
    print('Exiting')
    sys.exit()
result=qa({"query": user_input})
print("Response : ", result["result"])

Response :  The train number 128 comes at 01:12:00.


In [13]:
text = trans.translate(result["result"], src="en", dest=lang).text

print(text)

ರೈಲು ಸಂಖ್ಯೆ 128 01:12:00 ಕ್ಕೆ ಬರುತ್ತದೆ.


In [14]:
from gtts import gTTS
import os
from pydub import AudioSegment
import platform

# Create a gTTS object
tts = gTTS(text, lang=lang, slow=False)

# Save the audio file
tts.save("output.mp3")

# Play the audio using a media player
os.system("start output.mp3")  # For Linux
# os.system("start output.mp3")  # For Windows

audio = AudioSegment.from_mp3("output.mp3")

# Play the audio
audio.play()

C:\Users\navne\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
C:\Users\navne\AppData\Roaming\Python\Python310\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified